In [5]:
import pandas as pd

csv_url = "https://health-infobase.canada.ca/src/data/covidLive/covid19.csv"
df = pd.read_csv(csv_url)


In [6]:
#parse government data so it is usable
df['YMD'] = pd.to_datetime(df['date'], dayfirst = True)
df = df.drop(['date', 'prnameFR', 'pruid'], axis = 1)
#if provinces don't report active cases, assume they have 0, so the data will plot
df['numactive'].fillna(0.0, inplace = True)

#List of unique provinces, Canada, ordered alphabetically with Canada first
provnames = df.prname.unique()
provnames = list(provnames)
provnames.pop()
provnames.sort()
myorder = [2, 0, 1, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]
provnames = [provnames[i] for i in myorder]
dropnames = ['Canada', 'Nunavut', 'Yukon', 'Northwest Territories']
select_provs = [x for x in provnames if x not in dropnames]

#function to make province-wise dataset
def make_dataset(pnames):
    
    by_prov = pd.DataFrame()

    # Iterate through all the provinces
    for i in pnames:

        # Subset to the province
        sub = df[df['prname'] == i]

        # Add to the overall dataframe
        by_prov = by_prov.append(sub)
    
    return by_prov

#must pivot datasets for linear plot data so prov names are columns
def format_dataset(df, value):
    df_pivot = pd.pivot_table(df,
    columns = 'prname',
    index = 'YMD',
    values = value)
    
    df_pivot = df_pivot.rename_axis(None)

    return df_pivot

In [7]:
dataset = format_dataset(make_dataset(select_provs), 'numtoday')
index_list = pd.date_range(start = '2021-04-30', end = '2021-05-28', closed = 'right')
month_period = dataset.loc[index_list, :]
#print(month_period)
month_total = month_period.sum(axis = 0)
print(month_total)
#month_total.to_csv('data/29JAN_19FEB_totals.csv', encoding = 'utf8')

prname
Alberta                      35715.0
British Columbia             14099.0
Manitoba                     11420.0
New Brunswick                  266.0
Newfoundland and Labrador      217.0
Nova Scotia                   3070.0
Ontario                      65089.0
Prince Edward Island            21.0
Quebec                       19545.0
Saskatchewan                  5187.0
dtype: float64
